# Deploying ML Model on Azure Using Python

In [1]:
!pip install azureml-sdk

  Using cached humanfriendly-10.0-py2.py3-none-any.whl (86 kB)
     ---------------------------------------- 0.0/80.9 kB ? eta -:--:--
     ---------------------------------------- 80.9/80.9 kB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/141.4 kB ? eta -:--:--
     ------------------- ------------------- 71.7/141.4 kB 1.9 MB/s eta 0:00:01
     -------------------------------------- 141.4/141.4 kB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/85.4 kB ? eta -:--:--
     ---------------------------------------- 85.4/85.4 kB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/40.5 kB ? eta -:--:--
     ---------------------------------------- 40.5/40.5 kB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/55.5 kB ? eta -:--:--
     ---------------------------------------- 55.5/55.5 kB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/41.7 kB ? eta -:--:--
     -------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.11.2 requires protobuf<4,>=3.9.2, but you have protobuf 4.25.1 which is incompatible.
tensorflow-intel 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.25.1 which is incompatible.


   ---------------- ----------------------- 7.3/17.5 MB 3.1 MB/s eta 0:00:04
   ---------------- ----------------------- 7.3/17.5 MB 3.1 MB/s eta 0:00:04
   ---------------- ----------------------- 7.3/17.5 MB 3.1 MB/s eta 0:00:04
   ---------------- ----------------------- 7.3/17.5 MB 3.1 MB/s eta 0:00:04
   ---------------- ----------------------- 7.3/17.5 MB 3.1 MB/s eta 0:00:04
   ---------------- ----------------------- 7.3/17.5 MB 3.1 MB/s eta 0:00:04
   ---------------- ----------------------- 7.3/17.5 MB 3.1 MB/s eta 0:00:04
   ---------------- ----------------------- 7.3/17.5 MB 3.1 MB/s eta 0:00:04
   ---------------- ----------------------- 7.3/17.5 MB 3.1 MB/s eta 0:00:04
   ---------------- ----------------------- 7.3/17.5 MB 3.1 MB/s eta 0:00:04
   ---------------- ----------------------- 7.4/17.5 MB 2.6 MB/s eta 0:00:04
   ----------------- ---------------------- 7.6/17.5 MB 2.6 MB/s eta 0:00:04
   ------------------ --------------------- 7.9/17.5 MB 2.6 MB/s eta 0:00:04

In [3]:
import os
import json
import requests

from azureml.core import Workspace 
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

In [33]:
# Load the env variables
from dotenv import load_dotenv
load_dotenv()

True

In [34]:
sub_id = os.getenv('subscription_id')
resource_grp = os.getenv('resource_group')
workspace_name = os.getenv('workspace_name')

In [35]:
print(workspace_name)
print(resource_grp)

PracticingDeployment
TestGroup


### Creating Resource group and Workspace on Azure Portal

In [36]:
# Create a Workspace
ws = Workspace.create(name=workspace_name,
                      subscription_id=sub_id,
                      resource_group=resource_grp)

print(f'Workspace {workspace_name} created')


Deploying StorageAccount with name practicistorage0e8cbe546.
Deploying KeyVault with name practicikeyvault2cf753b8.
Deploying AppInsights with name practiciinsights27316f3c.
Deployed AppInsights with name practiciinsights27316f3c. Took 8.68 seconds.
Deployed KeyVault with name practicikeyvault2cf753b8. Took 24.34 seconds.
Deployed StorageAccount with name practicistorage0e8cbe546. Took 25.68 seconds.
Deploying Workspace with name PracticingDeployment.
Deployed Workspace with name PracticingDeployment. Took 42.82 seconds.
Workspace PracticingDeployment created


In [37]:
model_path = 'diabetes_model.pkl'

In [38]:
model_name = 'diabetes_prediction_model'

In [39]:
# Register the Model in Azure ML
registered_model = Model.register(workspace=ws, model_name=model_name, model_path=model_path)

Registering model diabetes_prediction_model


In [40]:
# Create a conda env
conda_env = Environment('my-conda-env')
conda_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

In [42]:
# Create Inference Config
inference_config = InferenceConfig(entry_script='score.py', environment=conda_env)

In [43]:
# Specify deployment configuration for aci
# ACI : Azure Container Instance
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

In [46]:
service = Model.deploy(workspace=ws,
                       name='diabetes-prediction-service-2',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config)
service.wait_for_deployment(show_output=True)

C:\Users\Kunjesh\AppData\Local\Temp\ipykernel_5968\2734396559.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-01-22 11:01:56+05:30 Registering the environment.
2024-01-22 11:01:57+05:30 Building image..
2024-01-22 11:12:23+05:30 Generating deployment configuration..
2024-01-22 11:12:35+05:30 Submitting deployment to compute.
2024-01-22 11:12:41+05:30 Checking the status of deployment diabetes-prediction-service-2.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [48]:
scoring_uri = service.scoring_uri
scoring_uri

'http://383f21b4-4d8e-43be-b74a-1b993c5ac3bb.eastus.azurecontainer.io/score'

#### Cleaning up all the created resources

In [ ]:
# Delete the Service
service_name = 'diabetes-prediction-service-2'

# Delete the Model
registered_model = ws.models[model_name]
registered_model.delete()

In [57]:
# Delete the workspace
ws.delete(delete_dependent_resources=True)